In [ ]:


# ---

# # 🧠 1️⃣ First: What Problem Was LangChain Facing?

# Before Runnables, LangChain had:

# * `LLMChain`
# * `SequentialChain`
# * `SimpleSequentialChain`
# * `RouterChain`
# * `ConversationChain`
# * Custom chains for everything…

# Each chain had:

# * Different input formats
# * Different output formats
# * Different ways of calling them

# 👉 Result:
# The library became large, messy, and hard for beginners (like most AI engineers in 2023).

# ---

# # 🚨 The Core Problem

# Every component behaved differently.

# Example:

# | Component | How You Call It             |
# | --------- | --------------------------- |
# | LLM       | `.predict()`                |
# | Prompt    | `.format()`                 |
# | Retriever | `.get_relevant_documents()` |
# | Chain     | `.run()`                    |

# Nothing was standardized.

# So connecting them required glue code.

# That’s bad software design.

# ---

# # 💡 The Big Idea: Standard Interface

# LangChain developers asked:

# > What if EVERYTHING behaved the same way?

# So they created:

# # 🔥 Runnable = A Standard Unit of Work

# A Runnable is simply:

# > Any object that has an `invoke(input)` method.

# That’s it.

# ---

# # 🎯 What That Means

# Now:

# | Component | How You Call It |
# | --------- | --------------- |
# | LLM       | `.invoke()`     |
# | Prompt    | `.invoke()`     |
# | Parser    | `.invoke()`     |
# | Retriever | `.invoke()`     |
# | Chain     | `.invoke()`     |

# Everything behaves the same.

# This is called:

# > Interface Standardization

# Very important software engineering concept.

# ---

# # 🧩 Why This Is Powerful

# Because now you can do this:

# ```python
# prompt | model | parser
# ```

# Why does this work?

# Because:

# * `prompt` is Runnable
# * `model` is Runnable
# * `parser` is Runnable

# They all follow same contract:

# ```python
# def invoke(input):
#     ...
# ```

# ---

# # 🏗 Software Architecture Perspective

# This follows:

# * OOP principle
# * Interface abstraction
# * Polymorphism
# * Composition over inheritance

# You don't need to know what the object is.
# You only care:

# > Does it implement `invoke()`?

# If yes → it can be chained.

# ---

# # 🧵 What Happens Internally

# When you write:

# ```python
# chain = prompt | model | parser
# ```

# LangChain creates something like:

# ```python
# class RunnableSequence:
#     def invoke(self, input):
#         output1 = prompt.invoke(input)
#         output2 = model.invoke(output1)
#         output3 = parser.invoke(output2)
#         return output3
# ```

# So `|` is just syntactic sugar for composition.

# ---

# # 🔀 Why RunnableParallel Works

# Because Runnable is standardized.

# When you write:

# ```python
# RunnableParallel({
#     "a": chain1,
#     "b": chain2
# })
# ```

# LangChain does:

# ```python
# {
#     "a": chain1.invoke(input),
#     "b": chain2.invoke(input)
# }
# ```

# Same interface → easy scaling.

# ---

# # 🌳 Why RunnableBranch Works

# Branch logic:

# ```python
# if condition(input):
#     return runnable1.invoke(input)
# else:
#     return runnable2.invoke(input)
# ```

# Again — everything is runnable.

# ---

# # 📦 Why This Fixed LangChain

# Before:

# * 20 different chain types
# * Confusing architecture

# After Runnables:

# * Everything = Runnable
# * Chains are just compositions
# * Much smaller core
# * Much easier mental model

# ---

# # 🧠 The Real Mental Model

# Think like this:

# > Runnable = function with superpowers

# But better than function because it also supports:

# * streaming
# * async
# * batching
# * tracing
# * retry
# * fallback

# All standardized.

# ---

# # 🏭 Real Production Impact

# Because everything is Runnable:

# You can add:

# ```python
# chain.with_retry()
# chain.with_fallback()
# chain.with_config()
# chain.stream()
# chain.batch()
# ```

# Without rewriting logic.

# This is production-level architecture.



In [ ]:
# Task-Specific Runnables (4:42): 
# These are core LangChain components like PromptTemplate, LLMs, Retrievers, 
# and Parsers that have been converted into runnables (4:55). 
# They have a specific purpose, such as helping to design prompts or interact with LLMs (5:15).


# Runnable Primitives (6:22):
#  These runnables act as fundamental building 
# blocks that help connect other task-specific runnables to create complex AI workflows (6:27).
# Examples:
#  RunnableSequence,
#  RunnableParallel, 
#  RunnableBranch,
#  RunnableLambda,
# RunnablePassthrough . 
# They orchestrate how different runnables interact, whether sequentially, 
# in parallel, or conditionally (6:41).

In [ ]:
# 7️⃣ With Config / Utility Wrappers

# These are enhancements you can apply to any Runnable:

# with_retry()

# Retry if model fails.

# with_fallback()

# Fallback to another model if first fails.

# batch()

# Process multiple inputs at once.

# stream()

# Stream tokens instead of full response.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence

load_dotenv()

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI()

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

chain = RunnableSequence(prompt1, model, parser, prompt2, model, parser)

print(chain.invoke({'topic':'AI'}))

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough

load_dotenv()

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI()

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

joke_gen_chain = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'explanation': RunnableSequence(prompt2, model, parser)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

print(final_chain.invoke({'topic':'cricket'}))

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableParallel

load_dotenv()

prompt1 = PromptTemplate(
    template='Generate a tweet about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a Linkedin post about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI()

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'tweet': RunnableSequence(prompt1, model, parser),
    'linkedin': RunnableSequence(prompt2, model, parser)
})

result = parallel_chain.invoke({'topic':'AI'})

print(result['tweet'])
print(result['linkedin'])


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableLambda, RunnablePassthrough, RunnableParallel

load_dotenv()

def word_count(text):
    return len(text.split())

prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI()

parser = StrOutputParser()

joke_gen_chain = RunnableSequence(prompt, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(word_count)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

result = final_chain.invoke({'topic':'AI'})

final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])

print(final_result)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableBranch, RunnableLambda

load_dotenv()

prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

model = ChatOpenAI()

parser = StrOutputParser()

report_gen_chain = prompt1 | model | parser

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>300, prompt2 | model | parser),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'Russia vs Ukraine'}))


